# An analysis of an IMDb data snapshot. What films are highly viewed / rated, and what new films should I watch?

## Information about the data:

The data is taken from https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset, downloaded as csv files and uploaded to my own Postgres database. Whilst csv files could be directly accessed and manipulated via Python (Pandas), I wanted to gain experience managing and querying data in a database. The data set was up to date as of 2020, and so this analysis is a reflection of the data at that time.

Whilst the full data set contains all movies listed on IMDb, my queries will more than likely only consist of movies where English is a listed language and where there are more than 5000 recorded votes. I understand there is some bias to these restrictions, but my reasoning is as follows. To the former point, I am an English speaker and am primarily interested in English spoken movies. This is not always the case, but upon querying the database there were far too many non-English spoken films, and this did not benefit me in my aims for this analysis. To the latter point, movie ratings were heavily skewed by a lack of votes. The average vote count across all movies was just over 9000. I understand that newer movies are heavily discriminated by using this metric, but it was more beneficial overall. I settled on including all movies that had at least 5000 votes.

## What this notebook is:

This notebook contains an exploratory analysis into what films are popular. It will also serve as a means for me to find new films to watch, by querying the data set for the common features amongst the films I currently enjoy. By this analysis, I hope to gain a better understanding into why films are popular / highly rated, as well as finding 'hidden gems' that appeal to my tastes.

## Code imports and helper function for querying the database:

In [33]:
import psycopg2
import pandas as pd
import matplotlib as plt
import configparser

# read database parameters from a config file
try:
    config = configparser.ConfigParser()
    config.read('config.ini')
    params = dict(config['DEFAULT'])
except:
    print('No config.ini file found.')

In [34]:
def query_db(query, params):
    '''Execute SQL queries passed to this function and return query results as data frame'''
    
    conn = None
    try:
        conn = psycopg2.connect(**params)
        df = pd.read_sql_query(query, conn)
        conn.close()
        return(df)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

## Analysis:

__*Where to start?*__
<br>Lets first take a look at the 10 highest rated films as a starting point.

In [32]:
top_10_query = '''
    select original_title, year, genre, duration, avg_vote, votes
    from movies
    where language like '%English%' and votes > 5000
    order by avg_vote desc
    limit 10;
'''

df = query_db(top_10_query, params)
df

,original_title,year,genre,duration,avg_vote,votes
0,The Shawshank Redemption,1994,Drama,142,9.3,2278845
1,The Godfather,1972,"Crime, Drama",175,9.2,1572674
2,The Godfather: Part II,1974,"Crime, Drama",202,9.0,1098714
3,The Dark Knight,2008,"Action, Crime, Drama",152,9.0,2241615
4,12 Angry Men,1957,"Crime, Drama",96,8.9,668473
5,Pulp Fiction,1994,"Crime, Drama",154,8.9,1780147
6,Schindler's List,1993,"Biography, Drama, History",195,8.9,1183248
7,The Lord of the Rings: The Return of the King,2003,"Action, Adventure, Drama",201,8.9,1604280
8,The Lord of the Rings: The Fellowship of the Ring,2001,"Action, Adventure, Drama",178,8.8,1619920
9,Forrest Gump,1994,"Drama, Romance",142,8.8,1755490
